# Decision Tree Classifier

## Training

In [2]:
import pandas as pd
import numpy as np
import math

In [3]:
# loading data and setting up

In [4]:
#load data
file_path = "./data/adult_train_data.csv"
data = pd.read_csv(file_path)

X = data[["age", "workclass", "fnlwgt", "education", "education_num", "marital_status", "occupation", "relationship", "race", "sex", "hours_per_week", "native_country"]]
Y = data[["income_more50K"]]

feature_dict = {col: X[col].unique().tolist() for col in X.columns}

X

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,hours_per_week,native_country
0,30-39,State-gov,<100K,Bachelors,>=10,Never-married,Adm-clerical,Not-in-family,White,Male,35-40,United-States
1,50-59,Self-emp-not-inc,<100K,Bachelors,>=10,Married-civ-spouse,Exec-managerial,Husband,White,Male,<35,United-States
2,30-39,Private,200K-300K,HS-grad,8--9,Divorced,Handlers-cleaners,Not-in-family,White,Male,35-40,United-States
3,50-59,Private,200K-300K,11th,<8,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,35-40,United-States
4,20-29,Private,>300K,Bachelors,>=10,Married-civ-spouse,Prof-specialty,Wife,Black,Female,35-40,Cuba
...,...,...,...,...,...,...,...,...,...,...,...,...
30156,20-29,Private,200K-300K,Assoc-acdm,>=10,Married-civ-spouse,Tech-support,Wife,White,Female,35-40,United-States
30157,40-49,Private,100K-200K,HS-grad,8--9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,35-40,United-States
30158,50-59,Private,100K-200K,HS-grad,8--9,Widowed,Adm-clerical,Unmarried,White,Female,35-40,United-States
30159,20-29,Private,200K-300K,HS-grad,8--9,Never-married,Adm-clerical,Own-child,White,Male,<35,United-States


In [5]:
# Defining our splitting functions

In [6]:
def split(X, Y, feature, val):
    mask_yes = X[feature] == val
    mask_not = X[feature] != val
    
    X_yes = X[mask_yes]
    Y_yes = Y[mask_yes]

    X_not = X[mask_not]
    Y_not = Y[mask_not]
    
    return (feature, val, X_yes, Y_yes, X_not, Y_not)


def split_everything(X, Y):
    splits = []
    for feature in feature_dict:
        for val in feature_dict[feature]:
            splits.append(split(X, Y, feature, val))

    return splits

In [7]:
# Defining our entropy and information gain functions

In [8]:
def entropy(Y):
    total = Y.shape[0]
    more50k = int(Y.sum().iloc[0])
    less50k = total-more50k

    if total != 0:
        P_more50k = more50k / total
        P_less50k = less50k / total
    else:
        P_more50k = 1
        P_less50k = 1

    if P_more50k == 0:
        P_more50k = 1       # log(1) = 0 so this causes this probability to be ignored
    if P_less50k == 0:
        P_less50k = 1       # log(1) = 0 so this causes this probability to be ignored

    return -P_more50k * math.log(P_more50k, 2) - P_less50k * math.log(P_less50k, 2)


def infogain(Y_entropy, n, Y_yes, Y_no):
    n_yes = Y_yes.shape[0]
    n_no = Y_no.shape[0]

    return Y_entropy - (n_yes / n)*entropy(Y_yes) - (n_no / n)*entropy(Y_no)


def all_infogains(Y, splits):
    all_the_infogains = []

    Y_entropy = entropy(Y)
    for split in splits:
        this_infogain = infogain(Y_entropy, Y.shape[0], split[3], split[5])
        info_tuple = (split[0], split[1], this_infogain)
        all_the_infogains.append(info_tuple)

    return all_the_infogains

In [9]:
# Building a node class that we will use to construct our decision tree

In [10]:
class Node:
    def __init__(self, split, feature, left, right, leaf: bool, label=None):
        self.split = split
        self.feature = feature
        self.left = left
        self.right = right
        self.leaf = leaf
        self.label = label  # only for leaves


    def get_split(self):
        return self.split
    

    def get_feature(self):
        return self.feature
    

    def get_left(self):
        return self.left
    

    def get_right(self):
        return self.right


    def get_leaf(self):
        return self.leaf
    

    def get_label(self):
        return self.label
    

    def print_tree(self, prefix="", left=True, max_length=None, line_count=[0]):
        if max_length is not None and line_count[0] >= max_length:
            return
        
        connector = "├── " if left else "└── "
        if self.leaf:
            print(prefix + connector + f"Leaf: {self.label}")
        else:
            print(prefix + connector + f"{self.feature}: {self.split}")
        
        line_count[0] += 1
        
        if self.left and (max_length is None or line_count[0] < max_length):
            self.left.print_tree(prefix + ("│   " if left else "    "), True, max_length, line_count)
        if self.right and (max_length is None or line_count[0] < max_length):
            self.right.print_tree(prefix + ("│   " if left else "    "), False, max_length, line_count)

In [11]:
# creating a recursive function that trains our data

In [12]:
def train(X, Y):

    if Y['income_more50K'].nunique() == 1:
        return Node(None, None, None, None, True, round((int(Y.sum().iloc[0]))/Y.shape[0]))

    splits = split_everything(X, Y)
    infogains = all_infogains(Y, splits)

    infogains_frame = pd.DataFrame(infogains, columns =['Feature', 'Value', 'Information Gain'])
    #infogains_frame.sort_values('Information Gain', ascending=False, inplace=True)
    biggest_infogain = infogains_frame.nlargest(1, 'Information Gain')
    split_by_value = biggest_infogain['Value'].iloc[0]
    split_data = [data_tuple for data_tuple in splits if data_tuple[1] == split_by_value][0]

    X_yes = split_data[2]
    Y_yes = split_data[3]
    X_not = split_data[4]
    Y_not = split_data[5]

    if Y_yes.shape[0] == 0 or Y_not.shape[0] == 0:
        return Node(None, None, None, None, True, round((int(Y.sum().iloc[0]))/Y.shape[0]))

    return Node(split_by_value, split_data[0], train(X_yes, Y_yes), train(X_not, Y_not), False)

In [13]:
decision_tree_depth_full = train(X,Y)

decision_tree_depth_full.print_tree(max_length=50)

├── marital_status: Married-civ-spouse
│   ├── education_num: >=10
│   │   ├── education: Some-college
│   │   │   ├── age: 20-29
│   │   │   │   ├── hours_per_week: <35
│   │   │   │   │   ├── fnlwgt: 200K-300K
│   │   │   │   │   │   ├── occupation: Adm-clerical
│   │   │   │   │   │   │   ├── relationship: Husband
│   │   │   │   │   │   │   │   ├── Leaf: 0
│   │   │   │   │   │   │   │   └── Leaf: 1
│   │   │   │   │   │   │   └── Leaf: 0
│   │   │   │   │   │   └── Leaf: 0
│   │   │   │   │   └── occupation: Other-service
│   │   │   │   │       ├── fnlwgt: 200K-300K
│   │   │   │   │       │   ├── workclass: Private
│   │   │   │   │       │   │   ├── relationship: Husband
│   │   │   │   │       │   │   │   ├── Leaf: 0
│   │   │   │   │       │   │   │   └── Leaf: 0
│   │   │   │   │       │   │   └── Leaf: 0
│   │   │   │   │       │   └── Leaf: 0
│   │   │   │   │       └── occupation: Machine-op-inspct
│   │   │   │   │           ├── fnlwgt: 200K-300K
│   │   │   │   │       

### Discussion
The full depth tree above is 11289 lines long, so it is presented truncated to the top 50 lines ease of viewing. The left branch (or "yes" branch) of each rule split is printed first with the right branch (or "no" branch) printed after.

Given the depth of this tree we would expect it to have a very high accuracy when classifying the training data, but it should be noted that as we only have 12 features per example (where each feature can have 2 - 40 different values) and 30161 examples, we will have a large number of identical examples (and indeed, exploring the raw csv confirms this). If any of these identical examples have different labels, these will be impossible for this model to tell apart. Hence, we would not expect 100% accuracy when classifying our training data but we would still expect out accuracy to be as high as is possible given the limitations of the data.

But of course, thanks to the fundamental trade off, we would expect the variance of our model to be very high, and hence this model will likely preform quite poorly on our test set thanks to overfitting the training set. A solution to this is to control the depth of our tree, which we explore in Part B.

## Controlling Depth

In [14]:
def train_with_depth(X, Y, max_depth, current_depth=0):

    if current_depth == max_depth:
        return Node(None, None, None, None, True, round((int(Y.sum().iloc[0]))/Y.shape[0]))

    if Y['income_more50K'].nunique() == 1:
        return Node(None, None, None, None, True, round((int(Y.sum().iloc[0]))/Y.shape[0]))

    splits = split_everything(X, Y)
    infogains = all_infogains(Y, splits)

    infogains_frame = pd.DataFrame(infogains, columns =['Feature', 'Value', 'Information Gain'])
    #infogains_frame.sort_values('Information Gain', ascending=False, inplace=True)
    biggest_infogain = infogains_frame.nlargest(1, 'Information Gain')
    split_by_value = biggest_infogain['Value'].iloc[0]
    split_data = [data_tuple for data_tuple in splits if data_tuple[1] == split_by_value][0]

    X_yes = split_data[2]
    Y_yes = split_data[3]
    X_not = split_data[4]
    Y_not = split_data[5]

    if Y_yes.shape[0] == 0 or Y_not.shape[0] == 0:
        return Node(None, None, None, None, True, round((int(Y.sum().iloc[0]))/Y.shape[0]))

    return Node(split_by_value, split_data[0], train_with_depth(X_yes, Y_yes, max_depth, current_depth+1), train_with_depth(X_not, Y_not, max_depth, current_depth+1), False)

In [15]:
decision_tree_depth_2 = train_with_depth(X, Y, 2)

decision_tree_depth_2.print_tree()

├── marital_status: Married-civ-spouse
│   ├── education_num: >=10
│   │   ├── Leaf: 1
│   │   └── Leaf: 0
│   └── hours_per_week: >40
│       ├── Leaf: 0
│       └── Leaf: 0


In [16]:
decision_tree_depth_3 = train_with_depth(X, Y, 3)

decision_tree_depth_3.print_tree()

├── marital_status: Married-civ-spouse
│   ├── education_num: >=10
│   │   ├── education: Some-college
│   │   │   ├── Leaf: 0
│   │   │   └── Leaf: 1
│   │   └── education_num: 8--9
│   │       ├── Leaf: 0
│   │       └── Leaf: 0
│   └── hours_per_week: >40
│       ├── education_num: >=10
│       │   ├── Leaf: 0
│       │   └── Leaf: 0
│       └── occupation: Prof-specialty
│           ├── Leaf: 0
│           └── Leaf: 0


In [17]:
decision_tree_depth_4 = train_with_depth(X, Y, 4)

decision_tree_depth_4.print_tree()

├── marital_status: Married-civ-spouse
│   ├── education_num: >=10
│   │   ├── education: Some-college
│   │   │   ├── age: 20-29
│   │   │   │   ├── Leaf: 0
│   │   │   │   └── Leaf: 0
│   │   │   └── education: Assoc-voc
│   │   │       ├── Leaf: 0
│   │   │       └── Leaf: 1
│   │   └── education_num: 8--9
│   │       ├── age: 20-29
│   │       │   ├── Leaf: 0
│   │       │   └── Leaf: 0
│   │       └── hours_per_week: >40
│   │           ├── Leaf: 0
│   │           └── Leaf: 0
│   └── hours_per_week: >40
│       ├── education_num: >=10
│       │   ├── age: 20-29
│       │   │   ├── Leaf: 0
│       │   │   └── Leaf: 0
│       │   └── age: 20-29
│       │       ├── Leaf: 0
│       │       └── Leaf: 0
│       └── occupation: Prof-specialty
│           ├── age: 20-29
│           │   ├── Leaf: 0
│           │   └── Leaf: 0
│           └── occupation: Exec-managerial
│               ├── Leaf: 0
│               └── Leaf: 0


### Discussion
Controlling the tree depth produces much smaller trees that are far faster to build. Continuing our discussion of the fundamental trade off from our previous discussion, we would expect our smaller trees to be better for generalisation to more data. These models are much less fitted to our training data specifically which we would give these models lower variance. Of course, for our especially shallow depths (e.g. our tree with depth 2), this depth control may not provide enough splits for good classification of data. We will next use classifier evaluation metrics to evaluate which models are the most effective. In future we may want to set depth as a hyper-parameter and experiment for a more optimum depth.

Something worth noting is that we have some branches that lead to two leaves with the same label. E.g. in our depth 4 tree, our bottom most split "occupation: Exec-managerial" has two leaves with label 0 (income not more than 50k). This creates a less-than-minimal tree where we could just equivalently turn the "occupation: Exec-managerial" node into a leaf with label 0.

## Evaluation

In [18]:
# Definining the functions that run new data through our models

In [19]:
def predict_example(current_node, example):
    if current_node.get_leaf():
        return current_node.get_label()
    
    split = current_node.get_split()
    if split in example:
        return predict_example(current_node.get_left(), example)
    return predict_example(current_node.get_right(), example)


def predict_all(X, decision_tree_root):
    Y_list = []
    for example in X.itertuples():
        Y_list.append(predict_example(decision_tree_root, list(example)))

    return pd.DataFrame({"income_more50K": Y_list})

In [20]:
# Functions for calculating our classifier evaluation metrics

In [21]:
def confusion(Y_predicted: list, Y_true: list):
    TN = 0
    TP = 0
    FP = 0
    FN = 0

    for i in range(len(Y_true)):
        if Y_true[i] == 0 and Y_predicted[i] == 0:
            TN += 1
        elif Y_true[i] == 1 and Y_predicted[i] == 1:
            TP += 1
        elif Y_true[i] == 0 and Y_predicted[i] == 1:
            FP += 1
        elif Y_true[i] == 1 and Y_predicted[i] == 0:
            FN += 1
    
    return (TN, TP, FP, FN)


def accuracy(confusion_tuple):
    TN = confusion_tuple[0]
    TP = confusion_tuple[1]
    FP = confusion_tuple[2]
    FN = confusion_tuple[3]
    return (TP + TN) / (TP + TN + FP + FN)


def precision(confusion_tuple):
    TN = confusion_tuple[0]
    TP = confusion_tuple[1]
    FP = confusion_tuple[2]
    FN = confusion_tuple[3]
    return (TP) / (TP + FP)


def recall(confusion_tuple):
    TN = confusion_tuple[0]
    TP = confusion_tuple[1]
    FP = confusion_tuple[2]
    FN = confusion_tuple[3]
    return (TP) / (TP + FN)


def f_measure(precision, recall, beta=1):
    return (1 + beta) * (precision*recall) / (beta**2 * precision + recall)

In [22]:
# Retrieving our test data and obtaining our evaluation metrics

In [23]:
file_path_test = "./data/adult_test_data.csv"
data_test = pd.read_csv(file_path_test)

X_test = data_test[["age", "workclass", "fnlwgt", "education", "education_num", "marital_status", "occupation", "relationship", "race", "sex", "hours_per_week", "native_country"]]
Y_test = data_test[["income_more50K"]]

Y_pred_depth_full = predict_all(X_test, decision_tree_depth_full)
Y_pred_depth_2 = predict_all(X_test, decision_tree_depth_2)
Y_pred_depth_3 = predict_all(X_test, decision_tree_depth_3)
Y_pred_depth_4 = predict_all(X_test, decision_tree_depth_4)

In [24]:
confusion_depth_full = confusion(list(Y_pred_depth_full["income_more50K"]), list(Y_test["income_more50K"]))
confusion_depth_2 = confusion(list(Y_pred_depth_2["income_more50K"]), list(Y_test["income_more50K"]))
confusion_depth_3 = confusion(list(Y_pred_depth_3["income_more50K"]), list(Y_test["income_more50K"]))
confusion_depth_4 = confusion(list(Y_pred_depth_4["income_more50K"]), list(Y_test["income_more50K"]))

depth_full_accuracy = accuracy(confusion_depth_full)
depth_full_precision = precision(confusion_depth_full)
depth_full_recall = recall(confusion_depth_full)
depth_full_f1_measure = f_measure(depth_full_precision, depth_full_recall)

depth_2_accuracy = accuracy(confusion_depth_2)
depth_2_precision = precision(confusion_depth_2)
depth_2_recall = recall(confusion_depth_2)
depth_2_f1_measure = f_measure(depth_2_precision, depth_2_recall)

depth_3_accuracy = accuracy(confusion_depth_3)
depth_3_precision = precision(confusion_depth_3)
depth_3_recall = recall(confusion_depth_3)
depth_3_f1_measure = f_measure(depth_3_precision, depth_3_recall)

depth_4_accuracy = accuracy(confusion_depth_4)
depth_4_precision = precision(confusion_depth_4)
depth_4_recall = recall(confusion_depth_4)
depth_4_f1_measure = f_measure(depth_4_precision, depth_4_recall)


print(f"{'Depth':<6}| {'Accuracy':<12}{'Precision':<12}{'Recall':<12}{'F1 Measure':<12}\n{'-' * 6}+{'-' * 49}")
print(f"{'Full':<6}| {depth_full_accuracy:<12.4f}{depth_full_precision:<12.4f}{depth_full_recall:<12.4f}{depth_full_f1_measure:<12.4f}")
print(f"{'2':<6}| {depth_2_accuracy:<12.4f}{depth_2_precision:<12.4f}{depth_2_recall:<12.4f}{depth_2_f1_measure:<12.4f}")
print(f"{'3':<6}| {depth_3_accuracy:<12.4f}{depth_3_precision:<12.4f}{depth_3_recall:<12.4f}{depth_3_f1_measure:<12.4f}")
print(f"{'4':<6}| {depth_4_accuracy:<12.4f}{depth_4_precision:<12.4f}{depth_4_recall:<12.4f}{depth_4_f1_measure:<12.4f}")

Depth | Accuracy    Precision   Recall      F1 Measure  
------+-------------------------------------------------
Full  | 0.7943      0.5894      0.5365      0.5617      
2     | 0.8066      0.6009      0.6341      0.6170      
3     | 0.8146      0.6735      0.4762      0.5579      
4     | 0.8170      0.7049      0.4389      0.5410      


### Discussion
We can see that for Accuracy our full depth tree is the worst performing model (at 79.4%), followed by our depth 2 tree, then depth 3, with our depth 4 tree performing the best (at 81.7%). This order is the same for our Precision metric, this time with 58.9% as our lowest and 70.5% as our highest. For Recall, we have our depth 4 tree at 43.9%, followed by depth 3, then full depth, giving us our depth 2 tree as our best performing model at 63.4%. For our F1 Measure we have the same order, with 54.1% as our lowest score and 61.7% as our highest.

Our Accuracy scores suggest that our model initially gets more accurate as depth increases, but there is a point where this switches, and exploration of depth as a hyper-parameter would be needed to give a better idea of where this point is. The Accuracy scores are also the highest of all our metrics, indicating that all our models are comparatively good at general classifying. We will further discuss this below. Our Precision scores show a similar story, indicating both that tentitive increases in depth may continue to result in better performance (but again we cannot be sure without further experimentation) and that when our models (particularly depths 3 & 4) classify positive labels they are indeed typically positive (i.e. low false positive rate). It follows then that all our models perform poorly at Recall, showing that we have a fairly high false negative rate.

# Summary

We used the information gain (infogain) criterion to decide on splits for our trees. This criterion bases splits purely on the infogain calculation, giving no additional preference to positive or negative labels. This makes this a good criterion to use when all we care is about classifying our data, with no preference for correctly classifying positives or negatives. The other splitting criterion we explored in lectures was "decision stump accuracy" or "classification accuracy" which decides on splits based on "if we use this rule, how many examples do we label correctly?". Like infogain, this is a criterion that you would use if you only care about correctly classifying the data. This is typically a poorer method of choosing splits as it gives no insight to how useful a split will be for generating more splits. Hence, we would expect the "classification accuracy" criterion to result in poorer performance from models using it.

If we had more context for what we were using the data for, another splitting criterion may be better. E.g. if we were calcuating the expected income from tax for everyone over the 50k income threshold, we may care more about reducing false positives as we may prefer to end up with more money than expected rather than less money. In this case we may wish to have some splitting criterion that weighs the information gained from classifying positive values higher, which would result in a higher Precision score for the resulting models.

As previously observed, our Accuracy increases as our model gets deeper for small depths, but is worse for our full depth tree. This is a strong indicator that our full depth tree is very overfitted to the training data. When we look at Recall, we see that our best performing model is our depth 2 tree, and that our score gets worse in our depth 3 and 4 trees. This shows that our model quickly becomes worse at identifying positive labels at increases depth, which may indicate that our models are starting to overfit to the training data even in these early depths. Given that the difference between the best and the worst Accuracy scores is only ~2.3% while the difference between the best and the worst Recall scores is ~19.5%, this indicates that our model starts overfitting quickly.